In [1]:
from keras import applications

# build the VGG16 network
model = applications.VGG16(include_top=False,
                           weights='imagenet')

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

C:\Users\HOU\Anaconda3\envs\tensorflow_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import backend as K

layer_name = 'block5_conv3'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, :, :, filter_index])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, input_img)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])



NameError: name 'input_img' is not defined

In [3]:
import numpy as np

# we start from a gray image with some noise
input_img_data = np.random.random((1, 3, img_width, img_height)) * 20 + 128.
# run gradient ascent for 20 steps
for i in range(20):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * step

NameError: name 'img_width' is not defined

In [4]:
from scipy.misc import imsave

# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

img = input_img_data[0]
img = deprocess_image(img)
imsave('%s_filter_%d.png' % (layer_name, filter_index), img)

NameError: name 'input_img_data' is not defined

In [7]:
'''Visualization of the filters of VGG16, via gradient ascent in input space.
This script can run on CPU in a few minutes.
Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function

from scipy.misc import imsave
import numpy as np
import time
from keras.applications import vgg16
from keras import backend as K

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

# save the result to disk
imsave('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0      

Current loss value: 720.2506
Current loss value: 795.9496
Current loss value: 861.2925
Current loss value: 930.44135
Current loss value: 995.8628
Current loss value: 1059.2382
Current loss value: 1121.7231
Current loss value: 1181.3796
Current loss value: 1242.0264
Current loss value: 1300.3896
Current loss value: 1360.3865
Current loss value: 1417.1199
Filter 8 processed in 3s
Processing filter 9
Current loss value: 15.928455
Current loss value: 44.915405
Current loss value: 78.65311
Current loss value: 125.48717
Current loss value: 173.12082
Current loss value: 250.49602
Current loss value: 292.28986
Current loss value: 345.76044
Current loss value: 405.44653
Current loss value: 449.52722
Current loss value: 472.15344
Current loss value: 539.0901
Current loss value: 576.16675
Current loss value: 605.1649
Current loss value: 664.87463
Current loss value: 698.04785
Current loss value: 737.23096
Current loss value: 780.2394
Current loss value: 804.9868
Current loss value: 856.39355
Filt

Current loss value: 600.27136
Current loss value: 627.1531
Current loss value: 662.9006
Current loss value: 686.7384
Filter 24 processed in 3s
Processing filter 25
Current loss value: 1.6725321
Current loss value: 13.10903
Current loss value: 33.720337
Current loss value: 59.589127
Current loss value: 87.2645
Current loss value: 112.86459
Current loss value: 147.88281
Current loss value: 182.07277
Current loss value: 225.37735
Current loss value: 263.4024
Current loss value: 302.75946
Current loss value: 338.3028
Current loss value: 373.16437
Current loss value: 416.48663
Current loss value: 461.4149
Current loss value: 495.1554
Current loss value: 532.4387
Current loss value: 565.0623
Current loss value: 604.0409
Current loss value: 642.2722
Filter 25 processed in 3s
Processing filter 26
Current loss value: 0.0
Filter 26 processed in 0s
Processing filter 27
Current loss value: 2.6141973
Current loss value: 6.420449
Current loss value: 16.062962
Current loss value: 22.29052
Current los

Current loss value: 140.15547
Current loss value: 236.04517
Current loss value: 336.07925
Current loss value: 412.88647
Current loss value: 489.2955
Current loss value: 549.897
Current loss value: 621.4392
Current loss value: 669.2274
Current loss value: 724.44824
Current loss value: 797.3771
Current loss value: 846.13586
Current loss value: 916.6455
Current loss value: 978.6079
Current loss value: 1046.2965
Current loss value: 1093.4265
Current loss value: 1163.9436
Current loss value: 1218.8574
Current loss value: 1288.388
Filter 42 processed in 3s
Processing filter 43
Current loss value: 7.8809834
Current loss value: 38.11741
Current loss value: 76.76776
Current loss value: 120.19275
Current loss value: 163.31601
Current loss value: 202.2742
Current loss value: 239.42651
Current loss value: 279.51965
Current loss value: 318.81097
Current loss value: 358.43103
Current loss value: 396.13687
Current loss value: 435.4677
Current loss value: 469.59918
Current loss value: 507.1198
Current

Current loss value: 464.23544
Current loss value: 494.43036
Current loss value: 523.5951
Current loss value: 554.4562
Filter 56 processed in 3s
Processing filter 57
Current loss value: 0.0
Filter 57 processed in 0s
Processing filter 58
Current loss value: 29.203514
Current loss value: 73.66841
Current loss value: 134.58965
Current loss value: 210.59769
Current loss value: 278.01974
Current loss value: 331.44775
Current loss value: 386.08746
Current loss value: 432.66537
Current loss value: 476.5744
Current loss value: 514.4418
Current loss value: 561.1521
Current loss value: 593.9709
Current loss value: 635.7694
Current loss value: 664.04535
Current loss value: 707.7135
Current loss value: 729.6282
Current loss value: 773.4429
Current loss value: 794.9553
Current loss value: 838.38806
Current loss value: 858.5176
Filter 58 processed in 3s
Processing filter 59
Current loss value: 48.82401
Current loss value: 149.0824
Current loss value: 233.82587
Current loss value: 308.97552
Current lo

...

Current loss value: 14.197994
Current loss value: 43.155743
Current loss value: 85.1759
Current loss value: 141.04205
Current loss value: 185.5094
Current loss value: 241.73581
Current loss value: 299.26154
Current loss value: 342.32117
Current loss value: 390.98987
Current loss value: 443.2804
Current loss value: 492.35095
Current loss value: 556.0208
Current loss value: 616.42145
Current loss value: 681.30225
Current loss value: 741.6205
Current loss value: 807.04266
Current loss value: 871.4928
Current loss value: 930.57587
Current loss value: 998.21735
Current loss value: 1063.2714
Filter 146 processed in 4s
Processing filter 147
Current loss value: 0.057369087
Current loss value: 11.25428
Current loss value: 49.24057
Current loss value: 122.74179
Current loss value: 180.47104
Current loss value: 227.76062
Current loss value: 268.52585
Current loss value: 309.2001
Current loss value: 353.61993
Current loss value: 388.8764
Current loss value: 437.37268
Current loss value: 476.27264


Current loss value: 557.7043
Current loss value: 591.21094
Current loss value: 624.7611
Current loss value: 659.73334
Current loss value: 694.1056
Current loss value: 728.6855
Filter 160 processed in 4s
Processing filter 161
Current loss value: 5.3994694
Current loss value: 31.968884
Current loss value: 84.537384
Current loss value: 146.58072
Current loss value: 214.73697
Current loss value: 264.49063
Current loss value: 317.2188
Current loss value: 364.7251
Current loss value: 410.23215
Current loss value: 454.31094
Current loss value: 500.06445
Current loss value: 530.4072
Current loss value: 573.5616
Current loss value: 610.7317
Current loss value: 645.78455
Current loss value: 686.3042
Current loss value: 722.17285
Current loss value: 757.645
Current loss value: 794.278
Current loss value: 830.52527
Filter 161 processed in 4s
Processing filter 162
Current loss value: 0.0
Filter 162 processed in 0s
Processing filter 163
Current loss value: 0.21390136
Current loss value: 0.49984464
C

Current loss value: 8.516525
Current loss value: 24.950817
Current loss value: 48.82846
Current loss value: 81.40404
Current loss value: 129.28673
Current loss value: 175.50702
Current loss value: 226.21234
Current loss value: 280.51663
Current loss value: 345.22815
Current loss value: 416.94705
Current loss value: 478.66782
Current loss value: 536.0597
Current loss value: 581.6107
Current loss value: 632.027
Current loss value: 672.0466
Current loss value: 725.0917
Current loss value: 776.5275
Current loss value: 824.34094
Current loss value: 875.5387
Current loss value: 922.55225
Filter 177 processed in 4s
Processing filter 178
Current loss value: 0.0
Filter 178 processed in 0s
Processing filter 179
Current loss value: 0.0
Filter 179 processed in 0s
Processing filter 180
Current loss value: 0.9304216
Current loss value: 16.00886
Current loss value: 38.911293
Current loss value: 63.659775
Current loss value: 89.46192
Current loss value: 122.50961
Current loss value: 145.6019
Current l

Current loss value: 357.51108
Current loss value: 393.18542
Current loss value: 430.57587
Current loss value: 463.10382
Current loss value: 503.9286
Current loss value: 535.7993
Current loss value: 574.0477
Current loss value: 609.9032
Current loss value: 645.9238
Current loss value: 677.0067
Filter 192 processed in 3s
Processing filter 193
Current loss value: 0.0
Filter 193 processed in 0s
Processing filter 194
Current loss value: 0.0
Filter 194 processed in 0s
Processing filter 195
Current loss value: 0.08860732
Current loss value: 1.0071356
Current loss value: 6.4135203
Current loss value: 14.1991825
Current loss value: 24.914328
Current loss value: 33.130623
Current loss value: 52.10925
Current loss value: 75.107635
Current loss value: 97.962975
Current loss value: 123.58926
Current loss value: 148.07016
Current loss value: 177.02582
Current loss value: 216.03307
Current loss value: 249.78032
Current loss value: 291.13284
Current loss value: 329.38403
Current loss value: 364.34155


C:\Users\HOU\Anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:133: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
